In [1]:
# 1. Load trace (waveform) which has sampling_rate = 200 Hz
from obspy.core import read
st = read("https://examples.obspy.org/ev0_6.a01.gse2")
st = st.select(component="Z")
tr = st[0]

In [2]:
# 2. Show information about this trace
df = tr.stats.sampling_rate
tr.stats

         network: 
         station: EV0_6
        location: 
         channel: EHZ
       starttime: 1970-01-01T01:00:00.000000Z
         endtime: 1970-01-01T01:00:59.995000Z
   sampling_rate: 200.0
           delta: 0.005
            npts: 12000
           calib: 1.0
         _format: GSE2
            gse2: AttribDict({u'instype': u'', u'datatype': u'CM6', u'hang': 0.0, u'auxid': u'', u'vang': -1.0, u'calper': 1.0})

In [3]:
# 3. Plot the time series!
tr.plot(type="relative")

In [4]:
# 4. Compute classic_sta_lta with STA=1000 samples (5s) and LTA=2000 samples (10s)
from obspy.signal.trigger import classic_sta_lta
cft = classic_sta_lta(tr.data, int(5*df), int(10*df))

In [ ]:
# 5. Plot the trace with corresponding STA/LTA ratio
from obspy.signal.trigger import plot_trigger
plot_trigger(tr, cft, 1.5, 0.5)

In [ ]:
# 6. Repeat 4 & 5, but with Z-detect
from obspy.signal.trigger import z_detect
help(z_detect)
cft = z_detect(tr.data, int(df*10))
plot_trigger(tr, cft, -0.4, -0.3)

In [ ]:
# 7. Repeat 4 & 5, but with recursive_sta_lta
from obspy.signal.trigger import recursive_sta_lta
cft = recursive_sta_lta(tr.data, int(5 * df), int(10 * df))
help('recursive_sta_lta')
plot_trigger(tr, cft, 1.2, 0.5)

In [ ]:
# 8. Repeat 4 & 5, but with carl_sta_trig (Earthworm)
from obspy.signal.trigger import carl_sta_trig
help('carl_sta_trig')
cft = carl_sta_trig(tr.data, int(df * 5), int(10 * df), 0.8, 0.8)
plot_trigger(tr, cft, 20.0, -20.0)

In [ ]:
# 9. Repeat 4 & 5, but with delayed_sta_lta
from obspy.signal.trigger import delayed_sta_lta
cft = delayed_sta_lta(tr.data, int(5*df), int(10*df))
plot_trigger(tr, cft, 5, 10)

In [ ]:
# 10. Let's try to optimize the STA & LTA settings for this data using the delayed STA:LTA
from obspy.signal.trigger import plot_trigger
from obspy.signal.trigger import classic_sta_lta
import random
m = -9990.0
MAX_SECONDS = int(tr.stats.npts / df) / 10
TSTART = 30.0
TEND = 40.0
for count in range(1,1000):
    # draw STA window length from uniform distribution of 0.1 to 10s
    sta_seconds = round(random.random()*10, 1) 
    # draw LTA window length uniform distribution of 2-10 times STA
    lta_seconds = sta_seconds * round(2+random.random()*8,0)
    #print sta_seconds, lta_seconds
    if lta_seconds < MAX_SECONDS:
        cft = classic_sta_lta(tr.data, int(sta_seconds * df), int(lta_seconds * df))
        if max(cft) > m:
            m = max(cft[int(df*TSTART):int(df*TEND)])
            sta_best = sta_seconds
            lta_best = lta_seconds
            print ("sta_seconds=%.1f lta_seconds=%.1f max(cft)=%.1f") % (sta_best, lta_best, m)
cft = classic_sta_lta(tr.data, int(sta_best * df), int(lta_best * df) )
plot_trigger(tr, cft, 3.5, 1.5)

In [ ]:
# 11. Tune / Optimize those setttings
from obspy.core import read
st = read("https://examples.obspy.org/ev0_6.a01.gse2")
st = st.select(component="Z")
tr = st[0]
import sys

sys.path.append('/Users/glennthompson')
import tune_sta_lta as tsl
tsl.tune_sta_lta(tr, 30.0, 40.0, 20.0, 30.0, 100)

tr.spectrogram()

In [ ]:
# 12. Try with more interesting data
# Pavlof explosions 2007 infrasound sensor at PN7A
import sys
sys.path.append('/Users/glennthompson')
import tune_sta_lta as tsl
from obspy.core import read
from obspy.core.utcdatetime import UTCDateTime
tstart = UTCDateTime(2007, 8, 28, 3, 55, 0)
tend = UTCDateTime(2007, 8, 28, 4, 0, 0)
st = read("/Users/glennthompson/PN7A.BDF.2007.240", starttime=tstart, endtime=tend)
tr = st[0]
tr.plot()

# Filter from 2-10 Hz
tr.filter('bandpass', freqmin=2.0, freqmax=10.0, corners=2, zerophase=True)
tr.plot(type='relative')

# Downsample from 100 Hz to 50 Hz (anti-aliasing filter applied)
tr.decimate(factor=2, strict_length=False)

# Run the tuning function
tsl.tune_sta_lta(tr, 130.0, 150.0, 20.0, 30.0, 100 )

tr.spectrogram()

# Create trigger events
from obspy.core.stream import Stream
st = Stream(traces=[tr])
from obspy.signal import coincidence_trigger
trig = coincidence_trigger('classicstalta', 2.5, 1, st, 1, sta=0.6, lta=6.0) 
from pprint import pprint
pprint(trig)

# Plot each trigger
pretrig = 5;
posttrig = 5;
for thistrig in trig:
    tr2 = tr.copy()
    tr2.trim(starttime = thistrig['time'] - pretrig, endtime = thistrig['time'] + thistrig['duration'] + posttrig) 
    tr2.plot()

infrasoundtr = tr.copy()

In [ ]:
# 13. Pavlof explosions 2007 Seismic sensor at PN7A
import sys
sys.path.append('/Users/glennthompson')
import tune_sta_lta as tsl
from obspy.core import read
from obspy.core.utcdatetime import UTCDateTime
tstart = UTCDateTime(2007, 8, 28, 3, 55, 0)
tend = UTCDateTime(2007, 8, 28, 4, 0, 0)
st = read("/Users/glennthompson/PN7A.EHZ.2007.240", starttime=tstart, endtime=tend)
seismictr = st[0]
seismictr.plot()

# Filter from 2-10 Hz
seismictr.filter('bandpass', freqmin=2.0, freqmax=10.0, corners=2, zerophase=True)
seismictr.plot(type='relative')

# Downsample from 100 Hz to 50 Hz (anti-aliasing filter applied)
seismictr.decimate(factor=2, strict_length=False)

# Run the tuning function
tsl.tune_sta_lta(seismictr, 130.0, 150.0, 20.0, 30.0, 100 )

seismictr.spectrogram()

## Now let's trigger on both channels
# Create trigger events
from obspy.core.stream import Stream
st = Stream(traces=[infrasoundtr, seismictr])
from obspy.signal import coincidence_trigger
trig = coincidence_trigger('classicstalta', 5.0, 1.5, st, 2, sta=0.6, lta=6.0) 
from pprint import pprint
pprint(trig)

# Plot each trigger
pretrig = 3;
posttrig = 3;
for thistrig in trig:
    st2 = st.copy()
    st2.trim(starttime = thistrig['time'] - pretrig, endtime = thistrig['time'] + thistrig['duration'] + posttrig) 
    st2.plot(equal_scale=False)

In [ ]:
# 14. Redoubt swarm March 22, 2009
import sys
sys.path.append('/Users/glennthompson')
import tune_sta_lta as tsl
from obspy.core import read
from obspy.core.utcdatetime import UTCDateTime
tstart = UTCDateTime(2009, 3, 22, 3, 55, 0)
tend = UTCDateTime(2009, 3, 22, 4, 0, 0)
st = read("/Users/glennthompson/R*2009:081:00:00:00", starttime=tstart, endtime=tend)
st.plot()

# Spectrogram
st.spectrogram()

# Filter from 0.8-12 Hz & downsample by factor 2
st.filter('bandpass', freqmin=0.8, freqmax=12.0, corners=2, zerophase=False)
st.decimate(factor=2, strict_length=False)
st.plot(type='relative')

# Run the tuning function
for tr in st:
    tsl.tune_sta_lta(tr, 270.0, 300.0, 20.0, 30.0, 30 )

# Create trigger events
from obspy.core.stream import Stream
st = Stream(traces=[tr])
from obspy.signal import coincidence_trigger
trig = coincidence_trigger('classicstalta', 2.5, 1, st, 1, sta=0.6, lta=6.0) 
from pprint import pprint
pprint(trig)

# Plot each trigger
pretrig = 5;
posttrig = 5;
for thistrig in trig:
    tr2 = tr.copy()
    tr2.trim(starttime = thistrig['time'] - pretrig, endtime = thistrig['time'] + thistrig['duration'] + posttrig) 
    tr2.plot()